In [311]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
import seaborn as sns

In [312]:
df= pd.read_csv('data_booking/booking_reviews.csv')
df.head()

,review_title,reviewed_at,reviewed_by,images,crawled_at,url,hotel_name,hotel_url,avg_rating,nationality,rating,review_text,raw_review_text,tags,meta
0,Exceptional,11 July 2021,Kyrylo,NaN,"07/20/2021, 01:09:27",https://www.booking.com/reviews/be/hotel/villa...,Villa Pura Vida,https://www.booking.com/hotel/be/villa-pura-vi...,9.7,Poland,10.0,"Everything was perfect! Quite, cozy place to r...","<p class=""review_pos ""><svg aria-label=""Positi...",Business trip~Solo traveller~Junior Suite~Stay...,"{'language': 'en-gb', 'source': 'https://www.b..."
1,I highly recommend this b&b! We enjoyed it a lot!,24 November 2019,Dimitri,NaN,"07/20/2021, 01:09:27",https://www.booking.com/reviews/be/hotel/villa...,Villa Pura Vida,https://www.booking.com/hotel/be/villa-pura-vi...,9.7,Belgium,9.0,Very friendly host and perfect breakfast!,"<p class=""review_pos ""><svg aria-label=""Positi...",Leisure trip~Couple~Deluxe Suite~Stayed 1 nigh...,"{'language': 'en-gb', 'source': 'https://www.b..."
2,Exceptional,3 January 2020,Virginia,NaN,"07/20/2021, 01:09:27",https://www.booking.com/reviews/be/hotel/hydro...,Hydro Palace Apartment,https://www.booking.com/hotel/be/hydro-palace....,9.2,United Kingdom,10.0,It was just what we wanted for a week by the b...,"<p class=""review_neg ""><svg aria-label=""Negati...",Leisure trip~Couple~Apartment with Sea View~St...,"{'language': 'en-gb', 'source': 'https://www.b..."
3,My stay in the house was a experiencing bliss ...,8 September 2019,Kannan,NaN,"07/20/2021, 01:09:28",https://www.booking.com/reviews/be/hotel/villa...,Villa Pura Vida,https://www.booking.com/hotel/be/villa-pura-vi...,9.7,Netherlands,10.0,My stay in the house was a experiencing bliss ...,"<p class=""review_pos ""><svg aria-label=""Positi...",Business trip~Solo traveller~Junior Suite~Stay...,"{'language': 'en-gb', 'source': 'https://www.b..."
4,One bedroom apartment with wonderful view and ...,23 June 2019,Sue,https://cf.bstatic.com/xdata/images/xphoto/squ...,"07/20/2021, 01:09:28",https://www.booking.com/reviews/be/hotel/hydro...,Hydro Palace Apartment,https://www.booking.com/hotel/be/hydro-palace....,9.2,South Africa,9.2,The building itself has a very musty smell in ...,"<p class=""review_neg ""><svg aria-label=""Negati...",Leisure trip~People with friends~Apartment wit...,"{'language': 'en-gb', 'source': 'https://www.b..."


In [313]:
display(df.info())
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26386 entries, 0 to 26385
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   review_title     26385 non-null  object 
 1   reviewed_at      26386 non-null  object 
 2   reviewed_by      26386 non-null  object 
 3   images           754 non-null    object 
 4   crawled_at       26386 non-null  object 
 5   url              26386 non-null  object 
 6   hotel_name       26386 non-null  object 
 7   hotel_url        26386 non-null  object 
 8   avg_rating       26386 non-null  float64
 9   nationality      26370 non-null  object 
 10  rating           26386 non-null  float64
 11  review_text      26386 non-null  object 
 12  raw_review_text  26386 non-null  object 
 13  tags             26386 non-null  object 
 14  meta             26386 non-null  object 
dtypes: float64(2), object(13)
memory usage: 3.0+ MB


None

review_title           1
reviewed_at            0
reviewed_by            0
images             25632
crawled_at             0
url                    0
hotel_name             0
hotel_url              0
avg_rating             0
nationality           16
rating                 0
review_text            0
raw_review_text        0
tags                   0
meta                   0
dtype: int64

In [314]:
#pre-select columns into the dataframe --> 
#even though we have included other attributes such as hotel_name, they are meant for competitor analysis
#the main priority is the review_text where topic modelling takes place 
cols = ['hotel_name','nationality','rating','review_text']
df_cols = df[cols]

# drop all the null rows
df_cols = df_cols.dropna()
# drop rows without comment
df_cols = df_cols[df_cols['review_text'] != "There are no comments available for this review"]

df_cols.head()

,hotel_name,nationality,rating,review_text
0,Villa Pura Vida,Poland,10.0,"Everything was perfect! Quite, cozy place to r..."
1,Villa Pura Vida,Belgium,9.0,Very friendly host and perfect breakfast!
2,Hydro Palace Apartment,United Kingdom,10.0,It was just what we wanted for a week by the b...
3,Villa Pura Vida,Netherlands,10.0,My stay in the house was a experiencing bliss ...
4,Hydro Palace Apartment,South Africa,9.2,The building itself has a very musty smell in ...


# Text-Processing 

In [315]:
import re
import string
import time
import nltk
from nltk.corpus import words
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('words')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/victor/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/victor/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/victor/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/victor/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to /home/victor/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

Sequence of text cleaning 
1. removing most used words 
2. remove punctuations and special characters 
3. lower casing 
2. stopwords -> the whole sentence becomes list 
3. lemmatization
6. tokenize 

In [316]:
def preprocess(txt):
    cleaned = txt
    # Remove the named entities
    name_entities_to_remove = [r'hotels?', r'rooms?', r"[Bb]&[Bb]'?s?"]
    for entity in name_entities_to_remove:
        # Use a non-letter before and after the entity to ensure whole-word matching
        #df_cols['review_text'] = df_cols['review_text'].astype(str).str.replace(f" {entity} ", " ")
        cleaned = re.sub(r'[^\w]' + entity + r'[^\w]', ' ', cleaned)
    
    #remove urls
    cleaned=re.sub(r'http\S+',' ',cleaned)
    #remove hashtags and word after hashtag
    cleaned=re.sub(r'#([^\s]+)',' ',cleaned)
    #remove emojis
    emoji=re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u'\U00010000-\U0010ffff'
        u"\u200d"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\u3030"
        u"\ufe0f"
                     "]+", flags = re.UNICODE)
    cleaned=emoji.sub(r'',cleaned)
    #remove punctuations and special characters
    cleaned=re.sub(r'[^\w\s]',' ',cleaned)
    cleaned="".join([c for c in cleaned if c not in string.punctuation])
    #remove numbers
    cleaned=re.sub('[0-9]+',' ',cleaned)
    #remove single character
    cleaned=re.sub(r'\s+[a-zA-Z]\s+',' ',cleaned)
    #to lower case
    cleaned=cleaned.lower()
    #remove newlines, multiple space
    cleaned=re.sub(r'\s+',' ',cleaned)
    #tokenized
    cleaned=cleaned.split()
    return cleaned

df_cols['review_text'] = df_cols['review_text'].map(lambda s:preprocess(s))

In [317]:
#Remove non-english words
english_word_set = set(words.words())
df_cols['review_text'] = df_cols['review_text'].apply(lambda x: [item for item in x if item in english_word_set or not item.isalpha()])

In [318]:
df_cols.head()

,hotel_name,nationality,rating,review_text
0,Villa Pura Vida,Poland,10.0,"[everything, was, perfect, quite, cozy, place,..."
1,Villa Pura Vida,Belgium,9.0,"[very, friendly, host, and, perfect, breakfast]"
2,Hydro Palace Apartment,United Kingdom,10.0,"[it, was, just, what, we, for, week, by, the, ..."
3,Villa Pura Vida,Netherlands,10.0,"[my, stay, in, the, house, was, bliss, in, lux..."
4,Hydro Palace Apartment,South Africa,9.2,"[the, building, itself, very, musty, smell, in..."


In [319]:
#remove stopwords
stop_words=nltk.corpus.stopwords.words('english')
df_cols['review_text'] = df_cols['review_text'].apply(lambda x: [item for item in x if item not in stop_words])

In [320]:
df_cols.head()

,hotel_name,nationality,rating,review_text
0,Villa Pura Vida,Poland,10.0,"[everything, perfect, quite, cozy, place, relax]"
1,Villa Pura Vida,Belgium,9.0,"[friendly, host, perfect, breakfast]"
2,Hydro Palace Apartment,United Kingdom,10.0,"[week, beach, winter, location, apartment, qui..."
3,Villa Pura Vida,Netherlands,10.0,"[stay, house, bliss, luxury, house, sheer, tou..."
4,Hydro Palace Apartment,South Africa,9.2,"[building, musty, smell, hallway, despite, bui..."


In [321]:
from collections import defaultdict, Counter
# Assuming df_cols['review_text'] is a list of lists
all_words = [item for sublist in df_cols['review_text'] for item in sublist]
# Count word frequency
word_counts = Counter(all_words)
# Sort the word counts in ascending order
sorted_word_counts = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)
# Display the word counts
for word, count in sorted_word_counts:
    print(f"{word}: {count}")


location: 6925
breakfast: 6448
good: 5661
staff: 5091
nice: 4473
great: 4348
clean: 4114
friendly: 3487
comfortable: 2749
bed: 2494
stay: 2274
everything: 2166
really: 2155
helpful: 2149
excellent: 2049
place: 1997
bathroom: 1986
would: 1985
nothing: 1891
well: 1703
could: 1684
also: 1668
us: 1649
perfect: 1632
close: 1617
one: 1604
parking: 1590
shower: 1563
small: 1538
bit: 1424
like: 1411
lovely: 1341
night: 1293
city: 1222
walk: 1204
area: 1188
time: 1162
little: 1151
quiet: 1142
spacious: 1132
station: 1070
host: 1065
beautiful: 1057
coffee: 1055
room: 1049
check: 1045
apartment: 1042
big: 1032
even: 963
restaurant: 934
get: 905
floor: 897
service: 892
reception: 891
bar: 888
amazing: 882
easy: 878
price: 875
quite: 874
old: 867
center: 838
water: 829
food: 827
next: 799
view: 792
day: 783
super: 780
much: 778
need: 775
people: 756
central: 745
free: 742
door: 740
back: 734
kitchen: 700
walking: 696
recommend: 675
house: 674
made: 671
modern: 658
large: 655
outside: 654
around: 64

sticky: 19
queue: 19
spectacular: 18
vicinity: 18
catch: 18
bottom: 18
refurbishment: 18
channel: 18
awake: 18
thus: 18
listed: 18
neighbor: 18
similar: 18
path: 18
fill: 18
ease: 18
cycle: 18
mosquito: 18
guide: 18
grab: 18
enjoying: 18
soviet: 18
computer: 18
peter: 18
among: 18
powerful: 18
flooded: 18
mall: 18
sadly: 18
cake: 18
worse: 18
freezing: 18
initially: 18
client: 18
suit: 18
establishment: 18
splendid: 18
residential: 18
hill: 18
sliding: 18
zero: 18
ensure: 18
vacation: 18
radiator: 18
hesitate: 17
highlight: 17
organic: 17
review: 17
plumbing: 17
slippery: 17
report: 17
socket: 17
refund: 17
supportive: 17
waffle: 17
struggle: 17
tray: 17
spare: 17
month: 17
produce: 17
memorable: 17
match: 17
picturesque: 17
throw: 17
coast: 17
switched: 17
august: 17
understood: 17
broke: 17
vegetarian: 17
spiral: 17
localization: 17
het: 17
firm: 17
sugar: 17
playground: 17
taxis: 17
driver: 17
impeccable: 17
nightmare: 17
regulate: 17
avenue: 17
garbage: 17
shelf: 17
plate: 17
featu

vertical: 3
community: 3
doubled: 3
seasoned: 3
criticize: 3
flair: 3
grade: 3
theres: 3
inspire: 3
sleek: 3
oak: 3
excellence: 3
eatable: 3
objective: 3
occupy: 3
basilica: 3
rebuilt: 3
mere: 3
steel: 3
jolly: 3
dear: 3
sewing: 3
economical: 3
exist: 3
polish: 3
illumination: 3
operation: 3
strictly: 3
diet: 3
pit: 3
wat: 3
rhubarb: 3
aloud: 3
learned: 3
billed: 3
lockable: 3
depart: 3
attentiveness: 3
prevention: 3
circular: 3
compartment: 3
monsieur: 3
unavoidable: 3
scoring: 3
mop: 3
speedy: 3
electronics: 3
profit: 3
industry: 3
scattered: 3
grout: 3
steady: 3
slope: 3
unhealthy: 3
hugely: 3
priceless: 3
western: 3
talented: 3
karaoke: 3
ally: 3
talkative: 3
behalf: 3
limiting: 3
critical: 3
decently: 3
lie: 3
treasure: 3
teens: 3
loyal: 3
goof: 3
tendency: 3
meanwhile: 3
outbreak: 3
slippy: 3
gross: 3
handful: 3
gear: 3
vase: 3
alternatively: 3
tremendously: 3
alive: 3
gated: 3
lemon: 3
midst: 3
sympathetically: 3
flooding: 3
act: 3
sandy: 3
speaker: 3
buzzing: 3
color: 3
profile

glide: 1
dawn: 1
mower: 1
yar: 1
exposure: 1
pandemy: 1
exploitation: 1
population: 1
inaudible: 1
labour: 1
robbery: 1
alarming: 1
irritation: 1
supreme: 1
estimate: 1
lotus: 1
purify: 1
spouse: 1
belt: 1
trusting: 1
cased: 1
preservation: 1
licking: 1
irksome: 1
screeching: 1
beech: 1
ensemble: 1
proposition: 1
diversified: 1
juicy: 1
supervise: 1
yoy: 1
pipework: 1
spine: 1
parrot: 1
sac: 1
novel: 1
taught: 1
pronounce: 1
ulterior: 1
rebuild: 1
downstream: 1
inbound: 1
outbound: 1
cancelation: 1
dancing: 1
geographical: 1
mort: 1
repulsive: 1
whisky: 1
brigade: 1
premiere: 1
bizarre: 1
buzzer: 1
quarry: 1
development: 1
cage: 1
mixture: 1
neatness: 1
insure: 1
tuning: 1
amino: 1
sealine: 1
arranger: 1
canoe: 1
algae: 1
distressing: 1
powder: 1
pants: 1
picnic: 1
victory: 1
utility: 1
ben: 1
spectacle: 1
entomologist: 1
overripe: 1
fusion: 1
glue: 1
shin: 1
grandchild: 1
evaluation: 1
pico: 1
predominantly: 1
purple: 1
resemble: 1
insider: 1
i̇t: 1
tye: 1
prestation: 1
glamour: 1
mos

mistaking: 1
awhile: 1
tumbler: 1
microclimate: 1
representation: 1
placard: 1
illuminating: 1
scooter: 1
relentless: 1
global: 1
enable: 1
career: 1
citizen: 1
epitome: 1
niceness: 1
admiring: 1
baldish: 1
jelly: 1
jane: 1
curled: 1
forwards: 1
illustrate: 1
ketchup: 1
patisserie: 1
conservative: 1
wholeheartedly: 1
chatelaine: 1
infrared: 1
seam: 1
unsettled: 1
lecture: 1
carving: 1
harmonious: 1
competition: 1
surgery: 1
forethought: 1
validate: 1
salient: 1
abut: 1
weve: 1
unpleasantly: 1
kon: 1
poolroom: 1
insightful: 1
liner: 1
armored: 1
sabbath: 1
someway: 1
rushing: 1
intolerant: 1
hooter: 1
bumper: 1
fiercely: 1
extraordinarily: 1
mason: 1
allied: 1
alike: 1
signpost: 1
ave: 1
dishcloth: 1
retired: 1
petty: 1
outing: 1
beamer: 1
enterprise: 1
bet: 1
projector: 1
pursue: 1
concentrated: 1
expressly: 1
static: 1
sustainable: 1
gusto: 1
integration: 1
bawl: 1
energetic: 1
caroline: 1
coping: 1
filth: 1
briefing: 1
reassure: 1
sate: 1
shelter: 1
crushing: 1
commemorate: 1
ly: 1
d

In [322]:
#text lemmatization
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)
#init the Wordnet Lemmatizer
lemmatizer = WordNetLemmatizer()

def lemmatize(s):
#for lemmatizing the word
     s = [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in s]
     return s
#columns containing a list of words 
df_cols['review_text'] = df_cols['review_text'].apply(lambda x: lemmatize(x))

In [323]:
df_cols.head()

,hotel_name,nationality,rating,review_text
0,Villa Pura Vida,Poland,10.0,"[everything, perfect, quite, cozy, place, relax]"
1,Villa Pura Vida,Belgium,9.0,"[friendly, host, perfect, breakfast]"
2,Hydro Palace Apartment,United Kingdom,10.0,"[week, beach, winter, location, apartment, qui..."
3,Villa Pura Vida,Netherlands,10.0,"[stay, house, bliss, luxury, house, sheer, tou..."
4,Hydro Palace Apartment,South Africa,9.2,"[building, musty, smell, hallway, despite, bui..."


In [324]:
text = df_cols['review_text'].apply(' '.join)
text = text.str.cat(sep= ' ')

text

'everything perfect quite cozy place relax friendly host perfect breakfast week beach winter location apartment quiet well spotless clean stay house bliss luxury house sheer touch beauty wrap luxury kindness care make remarkable stay building musty smell hallway despite built apartment good bearable hot water go every cold water come hot water come back checked thing like apartment view great apartment furnish modern style everything may need apartment clean free secure parking outside building parking lot downstairs garage always promptly sent via book messenger nothing beautiful home great comfortable bed hostess enough gracious welcome even give husband lift conference thank generous hospitality would stay many send owner fill everything great perfect location perfect clean everything need inside best best bath spacious enough lot bath electric car free charge nothing place perfect excellent place stay highly recommend stay extremely helpful one best stayed good romantic around corn

In [ ]:
df_cols.to_csv('pre-processed_topic_modelling.csv')

# Exploratory analysis - word cloud


In [ ]:
!pip install wordcloud

In [ ]:
# Import the wordcloud library
from wordcloud import WordCloud
# Join the different processed titles together.
long_string = ','.join(df_cols['review_text'].apply(lambda x: ' '.join(map(str, x))))
# Create a WordCloud object
wordcloud = WordCloud(background_color="white", max_words=5000, contour_width=3, contour_color='steelblue')
# Generate a word cloud
wordcloud.generate(long_string)
# Visualize the word cloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

# LDA

In [ ]:
from gensim import corpora, models

dictionary_LDA = corpora.Dictionary(df_cols['review_text'])
corpus = [dictionary_LDA.doc2bow(list_of_tokens) for list_of_tokens in df_cols['review_text']]
corpus

In [ ]:
def compute_coherence_values_lda(dictionary, corpus, texts, limit, start, step):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary,random_state=42)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
import gensim
from gensim.models.coherencemodel import CoherenceModel

model_list, coherence_values = compute_coherence_values_lda(dictionary=dictionary_LDA, corpus=corpus, texts=df_cols['review_text'], start=2, limit=11, step=1)


In [ ]:
# Show graph
limit=11; start=2; step=1;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.title('LDA')
plt.legend(("coherence_values"), loc='best')
plt.show()# Print the coherence scores

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
len(model_list)

In [ ]:
# Select the model and print the topics
optimal_model = model_list[8]
model_topics = optimal_model.show_topics(formatted=False)
optimal_model.print_topics(num_words=20)

In [ ]:
# Assuming model_list contains LDA models with different numbers of topics
optimal_model_index = 6

# Select the optimal LDA model
optimal_model = model_list[optimal_model_index]

# Get the topics and their words
topics = optimal_model.show_topics(num_words=20, formatted=False)

# Generate and plot word clouds for each topic
for topic_number, words in topics:
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(dict(words))
    
    plt.figure(figsize=(5, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(f"Topic {topic_number + 1} Word Cloud")
    plt.show()

# Final Run and Visualization

In [ ]:
!pip install pyLDAvis

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

In [ ]:
# Make an index to word dictionary.
temp = dictionary_LDA[0]  # This is only to "load" the dictionary.
id2word = dictionary_LDA.id2token

# Create an LDA model with the optimal number of topics (7)
model = gensim.models.ldamodel.LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=2000,
    alpha='auto',
    eta='auto',
    iterations=100,
    num_topics=7,  # Set the number of topics to the determined optimal value
    passes=8,
    eval_every=None
)

In [ ]:
pyLDAvis.enable_notebook()
lda_viz = gensimvis.prepare(model, corpus, dictionary_LDA, sort_topics=True)
pyLDAvis.display(lda_viz)

Explaination on coherence value 
1. https://www.baeldung.com/cs/topic-modeling-coherence-score#:~:text=We%20can%20use%20the%20coherence,words%20are%20to%20each%20other.
2. https://www.linkedin.com/advice/1/how-do-you-evaluate-coherence-perplexity
3. https://datascienceplus.com/evaluation-of-topic-modeling-topic-coherence/
4. https://www.dataknowsall.com/topicmodels.html
5. https://namyalg.medium.com/how-many-topics-4b1095510d0e

# Sentiment Analysis

In [325]:
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()

def compound_score(txt):
    return sia.polarity_scores(txt)["compound"]

## Sentiments
def sentiment(score):
    emotion = ""
    if score >= 0.5:
        emotion = "Positive"
    elif score <= -0.5:
        emotion = "Negative"
    else:
        emotion = "Neutral"
    return emotion
if type(df_cols['review_text'][0]) == list:
    df_cols['review_text'] = df_cols['review_text'].apply(' '.join)

comp_scores = df_cols["review_text"].astype("str").apply(compound_score)
df_cols["sentiment_score"] = comp_scores

## Applying Sentiment
df_cols["sentiment"] = df_cols["sentiment_score"].apply(sentiment)
df_cols.to_csv("sentiment.csv")


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/victor/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [328]:
df_cols.head(20)

,hotel_name,nationality,rating,review_text,sentiment_score,sentiment
0,Villa Pura Vida,Poland,10.0,everything perfect quite cozy place relax,0.7823,Positive
1,Villa Pura Vida,Belgium,9.0,friendly host perfect breakfast,0.7845,Positive
2,Hydro Palace Apartment,United Kingdom,10.0,week beach winter location apartment quiet wel...,0.5859,Positive
3,Villa Pura Vida,Netherlands,10.0,stay house bliss luxury house sheer touch beau...,0.9538,Positive
4,Hydro Palace Apartment,South Africa,9.2,building musty smell hallway despite built apa...,0.9117,Positive
5,Villa Pura Vida,United Kingdom,10.0,nothing beautiful home great comfortable bed h...,0.8504,Positive
6,Love Nest Versailles - Unique Louis XIV decora...,Poland,10.0,many send owner fill everything great perfect ...,0.9738,Positive
7,Gite Namaste,Netherlands,8.3,spacious enough lot bath electric car free charge,0.5106,Positive
9,Villa Pura Vida,United States,10.0,nothing place perfect excellent place stay hig...,0.8049,Positive
10,Love Nest Versailles - Unique Louis XIV decora...,Belgium,10.0,romantic around corner train station nice diff...,0.9042,Positive


In [327]:
import plotly.graph_objects as go

fig = go.Figure()

Ratings = [1,2,3,4,5,6,7,8,9,10]
round_rating = round(df_cols['rating'])
for rating in Ratings:
    fig.add_trace(go.Violin(x=round_rating[round_rating == rating],
                            y=df_cols['sentiment_score'][round_rating == rating],
                            name=rating,
                            box_visible=True,
                            meanline_visible=True))
fig.update_layout(
    title="Violin plot of Rating and Sentiment Score with box plot",
    xaxis_title="rating",
    yaxis_title="sentiment score",
    font=dict(
        family="Courier New, monospace",
        size=12,
        
    )
)
fig.show()